### <strong> Jack's Car Rental Problem </strong>
This notebook provides a solution for "Jack's Car Rental Problem" presented as "Example 4.2" in the "Reinforcement Learning: An Introduction, Second Edition" book by Sutton and Barto.

##### <strong> Problem Description </strong>
Jack manages two locations for a nationwide car rental company. Each day, some number of customers arrive at each location to rent cars. If Jack has a car available, he rents it out and is credited 10 dollars by the national company. If he is out of cars at that location, then the business is lost. Cars become available for renting the day after they are returned. To help ensure that cars are available where they are needed, Jack can move them between the two locations overnight, at a cost of 2 dollars per car moved. We assume that the number of cars requested and returned at each location are Poisson random variables. Suppose λ (parameter for poisson process) is 3 and 4 for rental requests at the first and second locations and 3 and 2 for returns. To simplify the problem slightly, we assume that there can be no more than 20 cars at each location (any additional cars are returned to the nationwide company, and thus disappear from the problem) and a maximum of five cars can be moved from one location to the other in one night. We take the discount rate to be γ = 0.9 and formulate this as a continuing finite MDP, where the time steps are days, the state is the number of cars at each location at the end of the day, and the actions are the net numbers of cars moved between the two locations overnight.

In [6]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [18]:
class Location:
    def __init__(self, capacity, lambda_request, lambda_return, epsilon = 0.005):
        self.capacity = capacity
        self.lambda_request = lambda_request
        self.lambda_return = lambda_return
        self.epsilon = epsilon
        self.request_seq = self.__request_count()
        self.return_seq = self.__return_count()

    def __poisson_pmf(x, lam):
        return lam**x * math.exp(-lam) / math.factorial(x)

    def __request_count(self):
        count = 0
        prob = Location.__poisson_pmf(count, self.lambda_request)
        res = []
        prob_sum = 0.0

        while prob < self.epsilon:
            count += 1
            prob = Location.__poisson_pmf(count, self.lambda_request)


        while prob >= self.epsilon:
            prob_sum += prob
            res.append((count, prob))
            count += 1
            prob = Location.__poisson_pmf(count, self.lambda_request)

        res = [(count, prob / prob_sum) for count, prob in res]

        return res

    def __return_count(self):
        count = 0
        prob = Location.__poisson_pmf(count, self.lambda_return)
        res = []
        prob_sum = 0.0

        while prob < self.epsilon:
            count += 1
            prob = Location.__poisson_pmf(count, self.lambda_return)
    
        while prob >= self.epsilon:
            prob_sum += prob
            res.append((count, prob))
            count += 1
            prob = Location.__poisson_pmf(count, self.lambda_return)

        res = [(count, prob / prob_sum) for count, prob in res]

        return res
    
    def request_count(self):
        return self.request_seq
    
    def return_count(self):
        return self.return_seq

In [19]:
class Environment:
    def __init__(self):
        self.actions = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
        self.states = []
        self.location1 = Location(20, 3, 3)
        self.location2 = Location(20, 4, 2)
        self.rental_price = 10.0
        self.move_cost = -2.0

        for i in range(self.location1.capacity + 1):
            for j in range(self.location2.capacity + 1):
                self.states.append((i, j))
    
    def get_states(self):
        return self.states

    def get_actions(self):
        return self.actions
    
    def __get_reward(self, rented1, rented2, action):
        return self.rental_price * (rented1 + rented2) + self.move_cost * abs(action)
    
    def get_nextState_reward_prob(self, state, action):

        for loc1_req_count, loc1_req_prob in self.location1.request_count():
            for loc2_req_count, loc2_req_prob in self.location2.request_count():
                for loc1_ret_count, loc1_ret_prob in self.location1.return_count():
                    for loc2_ret_count, loc2_ret_prob in self.location2.return_count():
                        i = min(state[0] - action, 20)
                        j = min(state[1] + action, 20)

                        rented1 = min(i, loc1_req_count)
                        rented2 = min(j, loc2_req_count)

                        i = min(max(i - loc1_req_count, 0) + loc1_ret_count, 20)
                        j = min(max(j - loc2_req_count, 0) + loc2_ret_count, 20)
                        
                        reward = self.__get_reward(rented1, rented2, action)
                        prob = loc1_req_prob * loc2_req_prob * loc1_ret_prob * loc2_ret_prob
                        yield (int(i), int(j)), reward, prob
    
    @staticmethod
    def is_valid(state, action):
        return ((state[0] - action) >= 0) and ((state[1] + action) >= 0)

In [20]:
class Agent:
    def __init__(self, env, discount = 0.9):
        self.env = env
        self.V = None
        self.policy = None
        self.discount = discount
        self.counter = 1

    def policy_iteration(self, theta = 0.01):
        self.V = np.zeros((self.env.location1.capacity + 1, self.env.location2.capacity + 1))
        self.policy = np.zeros((self.env.location1.capacity + 1, self.env.location2.capacity + 1))
        self.counter = 1

        while True:
            prev_V = np.copy(self.V)

            self.policy_evaluation(theta)
            
            stable_policy = self.policy_improvement()

            if np.greater(self.V, prev_V).all():
                print("State Value Improved!")
            else:
                print("State Value did not Improve!")

            if stable_policy:
                print("-------- Policy Iteration Done! --------")
                break
            else:
                print("-------- Policy Iteration " + str(self.counter) + " --------")

            self.save_state_value_function(".\\outputs\\state_value_functions\\V" + str(self.counter) + ".png")
            self.save_policy(".\\outputs\\policies\\p" + str(self.counter) + ".png")
            self.counter += 1 
            
    def policy_evaluation(self, theta):

        while True:
            delta = 0

            for s in self.env.get_states():
                v = self.V[s[0], s[1]]
                tmp = 0
                for next_s, reward, prob in self.env.get_nextState_reward_prob(s, self.policy[s[0], s[1]]):
                    tmp += prob * (reward + self.discount * self.V[next_s[0], next_s[1]])
                self.V[s[0], s[1]] = tmp
                delta = max(delta, abs(v - self.V[s[0], s[1]]))
            
            print("---- Delta: ", delta)
            if delta < theta:
                print("Policy Evaluation Done!")
                break

    def policy_improvement(self):
        policy_stable = True
        for s in self.env.get_states():
            old_action = self.policy[s[0], s[1]]

            best_action = old_action
            best_action_value = -1e5

            for action in self.env.get_actions():
                
                if self.env.is_valid(s, action):
                    action_value = 0
                    for next_s, reward, prob in self.env.get_nextState_reward_prob(s, action):
                        action_value += prob * (reward + self.discount * self.V[next_s[0], next_s[1]])
                
                    if action_value > best_action_value:
                        best_action = action
                        best_action_value = action_value
            
            self.policy[s[0], s[1]] = best_action
            
            if old_action != self.policy[s[0], s[1]]:
                policy_stable = False

        return policy_stable
    
    def save_state_value_function(self, filename):
        plt.imshow(self.V)
        plt.colorbar()
        plt.savefig(filename)
        plt.close("all")

    def save_policy(self, filename):
        plt.imshow(self.policy)
        plt.colorbar()
        plt.savefig(filename)
        plt.close("all")

In [21]:
env = Environment()
agent = Agent(env)
agent.policy_iteration(theta = 0.5)

---- Delta:  192.6885555256171
---- Delta:  133.50525838311967
---- Delta:  90.08831907557612
---- Delta:  67.38679951388798
---- Delta:  53.35272707893796
---- Delta:  41.450842698242354
---- Delta:  32.43920415783441
---- Delta:  25.82547378884925
---- Delta:  21.57533428475324
---- Delta:  18.149322658189703
---- Delta:  15.229272324815327
---- Delta:  12.749712327617658
---- Delta:  10.650876909956025
---- Delta:  8.87969908655623
---- Delta:  7.3894373719191435
---- Delta:  6.139060505224165
---- Delta:  5.09268842507754
---- Delta:  4.219107524316428
---- Delta:  3.4913252633089655
---- Delta:  2.886141277361844
---- Delta:  2.383729741677598
---- Delta:  1.9672374337703218
---- Delta:  1.6224046246470039
---- Delta:  1.3372149329251215
---- Delta:  1.1015779875872909
---- Delta:  0.9070464140867784
---- Delta:  0.7465667817193093
---- Delta:  0.614262837469596
---- Delta:  0.5052485441337353
---- Delta:  0.41546802816048967
Policy Evaluation Done!
State Value Improved!
-------- 